In [1]:
import pandas as pd
from custom_functions_scRBA import *
import shutil

In [2]:
#define default in and out folders
def_in = "input_base"
def_out = "model_base"

In [3]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_stoich = pd.read_excel('./'+def_out+'/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Ribosome efficiency (amino acids translated per active ribosome per second)
# (script will convert the unit to per hour) (obsolete / implement directly in GAMS file now)
# kribo = 10.5

# Dummy protein's length (assigned to be the median protein length)
#taken from protein_dummy_prot_calc_ctherm
NAA_dummy = 276

In [4]:
### Enzyme synthesis requirement coupled to metabolic reaction rate
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
eqn_list = []
eqn_list_bi = []        #equation list for running the bisection method
kapp_list = []
eqn_list_equality = []

for i in idx:

    #for debugging, comment out when not used
    #print("df_stoich: ",df_stoich.id[i][4:])
    
    lhs = "v('ENZLOAD-" + df_stoich.id[i][4:] + "') * " + "kapp('" + i + "')"
    rhs = "%mu% * v('" + i + "')"
    rhs_bi = "mu * v('" + i + "')"
    eqn_list.append(lhs + ' =g= ' + rhs + ';')
    eqn_list_bi.append(lhs + ' =g= ' + rhs_bi + ';')
    eqn_list_equality.append(lhs + ' =e= ' + rhs + ';')

    #for debugging, comment out when not used
    #print("i: ",i,"\teqn: ",eqn_list[-1],"\t#: ",len(eqn_list))
    
eqn_idx = ['EnzCap'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzCap'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]
eqn_list_bi = ['EnzCap'+str(i)+'.. ' + eqn_list_bi[i] for i in range(0, len(eqn_list_bi))]
eqn_list_equality = ['EnzCap'+str(i)+'.. ' + eqn_list_equality[i] for i in range(0, len(eqn_list_equality))]

#write constraints file
with open('./'+def_out+'/RBA_enzCapacityConstraints_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))

#write constraint equations file
with open('./'+def_out+'/RBA_enzCapacityConstraints_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))

#write constraint equations file
with open('./'+def_out+'/RBA_enzCapacityConstraints_eqns_bi.txt', 'w') as f:
    f.write('\n'.join(eqn_list_bi))
    
#write equality constraints file
with open('./'+def_out+'/RBA_enzCapacityConstraints_eqns_equality_version.txt', 'w') as f:
    f.write('\n'.join(eqn_list_equality))

In [5]:
### Write prosyn reaction
idx = [i for i in df_stoich.index if i[:7] == 'PROSYN-']
prosyn = ["'" + i + "'" for i in idx]
prosyn = ['/'] + prosyn + ['/']
with open('./'+def_out+'/RBA_rxns_prosyn.txt', 'w') as f:
    f.write('\n'.join(prosyn))

In [6]:
#shutil.rmtree('../GAMS/'+def_out)
#shutil.copytree('./'+def_out+'/',"../GAMS/"+def_out+"/")

FileExistsError: [Errno 17] File exists: '../GAMS/model_base/'